# Load and Clean the data
* Filter out rows with missing data in the required columns (REQUIRED_COLS)
* Keep only entries that are not marked as retracted
* Remove duplicates
* Save cleaned data

## Load the data
Data was [downloaded from OpenAlex](https://openalex.org/works?page=1&filter=default.search%3ATopic%20Network%20Graph,default.search%3ABibliometrics,default.search%3A%22Machine%20Learning%22%20OR%20%22Large%20Language%20Model%22,type%3Atypes%2Farticle,publication_year%3A2019-2024,language%3Alanguages%2Fen&group_by=type,authorships.institutions.lineage,primary_topic.id,open_access.is_oa,publication_year,authorships.author.id,keywords.id,authorships.countries,institutions.is_global_south,best_oa_location.license,primary_location.source.publisher_lineage,is_retracted,authorships.institutions.continent,corresponding_author_ids,primary_topic.domain.id,primary_topic.field.id,grants.funder,open_access.any_repository_has_fulltext,primary_location.source.is_oa,has_doi,best_oa_location.is_accepted,corresponding_institution_ids,has_orcid,authorships.institutions.type,primary_location.source.id,best_oa_location.is_published,language,repository,primary_location.source.is_in_doaj).

In [1]:
import pandas as pd
import zipfile as zf
import pickle
import os

In [2]:
DATA_PATH = 'data'  # path to folder with (downloaded or generated) data, will not be committed to git
ACRONYM = 'openalex-tng'  # Acronym, aka machine readable name, of this dataset ('tng' stands for "topic netework graph")

In [3]:
data_df = pd.read_csv(os.path.join(DATA_PATH, f"{ACRONYM}.csv"), index_col=0)
print(f"The downloaded data set has {data_df.shape[0]} rows and {data_df.shape[1]} columns.")

The downloaded data set has 1349 rows and 173 columns.


## Clean the data

### Filter out rows with missing data in the required columns (REQUIRED_COLS)

In [4]:
# columns used in the analysis
USED_COLS = [
    'authorships.author.display_name', 'publication_year', 'title', 'primary_location.source.display_name','doi', 
    'has_fulltext', 'is_retracted',
    'abstract', 'open_access.is_oa', 'authorships.countries']
# columns required to have a value
REQUIRED_COLS = [val for val in USED_COLS if val not in ['doi', 'open_access.is_oa', 'authorships.countries']]

In [5]:
# First count rows with missing data in the required columns (REQUIRED_COLS)
clean_df = data_df
for col in REQUIRED_COLS:
    idx = clean_df[col].isna()
    print(f"Detected {idx.sum()} entries without {col}.")

# Filter out rows with missing data in the required columns (REQUIRED_COLS)
clean_df = data_df
for col in REQUIRED_COLS:
    idx = clean_df[col].isna()
    clean_df = clean_df[~idx]


Detected 5 entries without authorships.author.display_name.
Detected 0 entries without publication_year.
Detected 0 entries without title.
Detected 42 entries without primary_location.source.display_name.
Detected 0 entries without has_fulltext.
Detected 0 entries without is_retracted.
Detected 55 entries without abstract.


### Remove retracted articles

In [6]:
# remove retracted articles
idx = clean_df['is_retracted']
clean_df = clean_df[~idx]
print(f"Count of articles that have been retacted: {idx.sum()}")

Count of articles that have been retacted: 2


### Keep only articles with full text

In [7]:
# remove articles without fulltext
idx = clean_df['has_fulltext']
clean_df = clean_df[idx]
print(f"Count of articles that do not have full text: {(~idx).sum()}")

Count of articles that do not have full text: 1


### Drop duplicates
Duplicates are identified based on the values in the used columns (USED_COLS).

In [8]:
# remove duplicated rows
print(f"Found {clean_df.duplicated(subset=USED_COLS).sum()} duplicated rows")
clean_df.drop_duplicates(subset=USED_COLS, inplace=True)

Found 2 duplicated rows


In [9]:
print(f"After cleanup, the dataset has {clean_df.shape[0]} rows and {clean_df.shape[1]} columns")

After cleanup, the dataset has 1249 rows and 173 columns


## Save cleaned data

In [10]:
def save_dataset(filename, path, df):
    """Writes a zipped file with a correctly named csv file inside."""
    with zf.ZipFile(path, 'w') as ziparchive:
        ziparchive.writestr(filename, df.to_csv())

csvname = f"{ACRONYM}-clean.csv"
path = os.path.join(DATA_PATH, f"{csvname}.zip")
save_dataset(csvname, path, clean_df)